In [157]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import h2o

In [158]:
project_name = 'Dbeaver'
version_name = '21.1.0'

project_name = 'Redisson'
version_name = 'redisson-3.16.0'

n_cluster_divisible_arr = [5,7,10,15,20,25]
affinity_arr = ['euclidean', 'manhattan', 'cosine']
linkage_arr = ['complete', 'average', 'single']

n_cluster = 300
affinity = 'cosine'
linkage = 'single'

In [159]:
depends_dir = f'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_depends/{project_name}/{project_name}_{version_name}.json'
rootdir = f'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/{project_name}/{project_name}_{version_name}'

In [160]:
print(rootdir)

C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/Redisson/Redisson_redisson-3.16.0


In [161]:
full_dir_arr = []
for root, dirs, files in os.walk(rootdir):
    #print(root)
    #print(dirs)
    for element in files:
        if '.java' in element:
            dir_string = root + '\\' + element
            full_dir_arr.append(dir_string)

cluster_dict = {}
cluster_tree = {}


for element in full_dir_arr:
    element = element.split('\\')
    child = element[-1]
    parent = element[-2]
    cluster_tree[child] = parent

In [162]:
cluster_tree

{'ElementsSubscribeService.java': 'redisson',
 'JndiRedissonFactory.java': 'redisson',
 'MapWriteBehindTask.java': 'redisson',
 'MapWriterTask.java': 'redisson',
 'PubSubEntry.java': 'handler',
 'PubSubMessageListener.java': 'redisson',
 'PubSubPatternMessageListener.java': 'redisson',
 'PubSubPatternStatusListener.java': 'redisson',
 'PubSubStatusListener.java': 'redisson',
 'QueueTransferService.java': 'redisson',
 'QueueTransferTask.java': 'redisson',
 'RedisClusterNodes.java': 'redisson',
 'RedisNodes.java': 'redisnode',
 'Redisson.java': 'redisson',
 'RedissonAtomicDouble.java': 'redisson',
 'RedissonAtomicLong.java': 'redisson',
 'RedissonBaseAdder.java': 'redisson',
 'RedissonBaseLock.java': 'redisson',
 'RedissonBatch.java': 'redisson',
 'RedissonBinaryStream.java': 'redisson',
 'RedissonBitSet.java': 'redisson',
 'RedissonBlockingDeque.java': 'redisson',
 'RedissonBlockingQueue.java': 'redisson',
 'RedissonBloomFilter.java': 'redisson',
 'RedissonBoundedBlockingQueue.java': 'r

In [163]:
len(cluster_tree)

1295

In [164]:
try:
    os.mkdir('raw_depends/' + project_name)
except:
    pass

command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & ' + f"java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/{project_name}/{project_name}_{version_name}" +  f' ../raw_depends/{project_name}/{project_name}_{version_name}' 
print(command)
os.system(command)

cd C:/Users/tanji/Desktop/SoftwareRemodularization/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/Redisson/Redisson_redisson-3.16.0 ../raw_depends/Redisson/Redisson_redisson-3.16.0


0

In [165]:
with open(depends_dir) as f:
    depends_results = json.load(f)
    index  = 0
    var_array = []
    print(len(depends_results['variables']))
    for value in depends_results['variables']:
        var_array.append([index, value.split('\\')[-1]])
        #print(index, value)
        index += 1

1849


In [166]:
with open(depends_dir) as f:
    depends_results = json.load(f)
    index  = 0
    var_array = []
    print(len(depends_results['variables']))
    print(len(depends_results['cells']))
    for value in depends_results['variables']:
        var_array.append([index, value.split('\\')[-1]])
        #print(index, value)
        index += 1

    var_df = pd.DataFrame(var_array)
    var_df.columns = ['index_val', 'name']

    feature_list = {}
    feature_index = 2
    for element in depends_results['cells']:
        #print(element)
        try:
            for a in element['values']:
                if a not in feature_list:
                    feature_list[a] = feature_index
                    feature_index += 1
                #print(a['Call'])


        except:
            pass

    feature_arr = []
    for element in depends_results['cells']:
        #print(array)
        array = [0] * (len(feature_list) + 2)
        values = dict(element['values'])
        #print(element)
        array[0] = element['src']
        array[1] = element['dest']
        for feature in feature_list:
            try:
                value = values[feature]
                array[feature_list[feature]] = value
            except:
                pass
        #print(array)
        feature_arr.append(array)

    feature_df = pd.DataFrame(feature_arr)
    col_names = ['src', 'dest']
    for element in feature_list:
        col_names.append(element)
    feature_df.columns = col_names

    feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']
    #G = nx.Graph()
    G = nx.path_graph(len(depends_results['variables']))
    for index, row in feature_df.iterrows():
        G.add_edge(row['src'], row['dest'], weight=row['sum'])

    adj_mat = nx.adjacency_matrix(G)
    adj_mat_df = pd.DataFrame(adj_mat.toarray())
    np.fill_diagonal(adj_mat_df.values, adj_mat_df.values.max())
    x = adj_mat_df.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    adj_mat_df = pd.DataFrame(x_scaled)
    
    sum_dependency_df = nx.to_pandas_adjacency(G)
    sum_dependency_df['sum'] = sum_dependency_df.sum(axis=1)
    final_dependency_df = sum_dependency_df['sum']

1849
32182


In [167]:
final_dependency_df

0       10.0
1       26.0
2        7.0
3       80.0
4        3.0
        ... 
1844    76.0
1845    32.0
1846     9.0
1847    66.0
1848    39.0
Name: sum, Length: 1849, dtype: float64

In [168]:
adj_mat_df

,0,1,2,3,4,5,6,7,8,9,...,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848
0,1.00000,0.00189,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00189,1.00000,0.00189,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.00000,0.00189,1.00000,0.00189,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.00000,0.00000,0.00189,1.00000,0.00189,0.00189,0.00189,0.00189,0.00189,0.00189,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.00000,0.00000,0.00000,0.00189,1.00000,0.00189,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1844,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.0,0.00189,1.00000,0.00189,0.00000,0.00000,0.00000
1845,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00189,0.0,0.0,0.00000,0.00189,1.00000,0.00189,0.00000,0.00000
1846,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00189,0.0,0.0,0.00000,0.00000,0.00189,1.00000,0.00189,0.00000
1847,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.00189,1.00000,0.00189


In [169]:
feature_df

,src,dest,Import,Use,Call,Parameter,Contain,Return,Create,Implement,Cast,Throw,Extend,Annotation,sum
0,288,1703,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,288,1702,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,288,1705,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,288,1704,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,288,1701,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32177,1569,932,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
32178,748,820,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
32179,1569,931,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
32180,1569,934,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
print('Currently processing: ' + project_name + version_name)
command = f'java -jar ck-0.3.3-SNAPSHOT-jar-with-dependencies.jar raw_sourcecode/{project_name}/{project_name}_{version_name}'
print(command)
os.system(command)
os.rename('class.csv', project_name + '_' + version_name + '_class.csv')
os.rename('field.csv',  project_name + '_' + version_name + '_field.csv')
os.rename('method.csv', project_name + '_' + version_name + '_method.csv')

shutil.move(project_name + '_' + version_name + '_class.csv', 'ck_results_invi/' + project_name + '_' + version_name + '_class.csv')
shutil.move(project_name + '_' + version_name + '_field.csv', 'ck_results_invi/' + project_name + '_' + version_name + '_field.csv')
shutil.move(project_name + '_' + version_name + '_method.csv', 'ck_results_invi/' + project_name + '_' + version_name + '_method.csv')

          

Currently processing: Redissonredisson-3.16.0
java -jar ck-0.3.3-SNAPSHOT-jar-with-dependencies.jar raw_sourcecode/Redisson/Redisson_redisson-3.16.0


'ck_results_invi/Redisson_redisson-3.16.0_method.csv'

In [170]:
try:
    os.mkdir('MoJo_1.2.1/' + project_name)
except:
    pass

In [171]:
cluster = AgglomerativeClustering(n_clusters=n_cluster, affinity=affinity, linkage=linkage)
cluster_result = cluster.fit_predict(adj_mat_df)

ck_metrics = pd.read_csv(f'ck_results_invi/{project_name}_{version_name}_class.csv')
ck_metrics['class_name'] = ck_metrics['file'].str.split('\\').str[-1]
filename_a = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + str(n_cluster) + '_' +str(affinity) + '_' + str(linkage) + '_a.rsf'
filename_b = 'C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1/' + project_name + '/' + project_name + '_' + version_name + '_' + str(n_cluster) + '_' +str(affinity) + '_' + str(linkage) + '_b.rsf'

with open(filename_a, 'w') as f:
    for key, value in cluster_tree.items():
        f.write('contain ' + str(value).replace(" ",'') + ' ' + str(key).replace(' ','') + '\n')

arr_a_rsf = []
arr_b_rsf = []
with open(filename_b, 'w') as f:
    len_b_rsf = 0
    duplicate_array = []
    refactor_arr = []
    main_data_arr = []
    clustering_result_dict = {}
    num_line_affected = 0
    num_dependency = 0
    actual_num_of_classes_touched = 0
    for i in range(len(cluster_result)):
        try:
            subject = var_df[var_df['index_val'] == i]['name'].values[0]
            print(subject)
            #print(subject in cluster_tree)
            if subject in cluster_tree and subject not in duplicate_array:
                print(element)
                duplicate_array.append(subject)
                string = "contain " + str(cluster_result[i]) + " " + subject + "\n"
                arr_b_rsf.append(subject)
                len_b_rsf += 1
                f.write(string)
                #print(subject)
                class_metrics = ck_metrics.loc[ck_metrics['class_name'] == subject].values.tolist()[0]
                
                #print(ck_metrics.loc[ck_metrics['class_name'] == subject]['file'].values[0])
                
                subjected_file = ck_metrics.loc[ck_metrics['class_name'] == subject]['file'].values[0]
                
                #print(subjected_file)
                
                num_dependency = final_dependency_df.iloc[cluster_result[i]]
                num_line_affected += sum(1 for line in open(subjected_file))
                
                
                class_metrics.append(num_dependency)
                class_metrics.append(num_line_affected)
                main_data_arr.append(class_metrics)
                
                
                ## Important
                ## This is assuming that during Depends, the file structure is already sorted
                ## I.e. going through the directory structure depth wise, not breath wise
                original_parent = cluster_tree[subject]
                try:
                    to_check_parent = clustering_result_dict[cluster_result[i]]
                    print(original_parent, to_check_parent)
                    if original_parent != to_check_parent:
                        refactor_arr.append(f'Move {subject} to {str(cluster_result[i])}')
                        actual_num_of_classes_touched += 1
                    
                    clustering_result_dict[cluster_result[i]] = to_check_parent
                except:
                    clustering_result_dict[cluster_result[i]] = original_parent
                
            else:
                #print(subject)
                pass
        except Exception as e:
            print(e)
            pass
f.close()

command = 'cd C:/Users/tanji/Desktop/SoftwareRemodularization/MoJo_1.2.1 & ' + 'java MoJo ' + filename_a + ' ' + filename_b + ' >> ' + project_name + '/' + project_name + '_results.txt'
#print(command)
os.system(command)

RedissonExtension.java
Annotation
RedissonExtensionTest.java
Annotation
redisson helidon
JndiRedissonRegionFactory.java
Annotation
hibernate helidon
RedissonRegionFactory.java
Annotation
hibernate helidon
RedissonStrategyRegistrationProvider.java
Annotation
BaseRegion.java
Annotation
region helidon
RedissonCollectionRegion.java
Annotation
region helidon
RedissonEntityRegion.java
Annotation
region helidon
RedissonNaturalIdRegion.java
Annotation
region helidon
RedissonQueryRegion.java
Annotation
region helidon
RedissonTimestampsRegion.java
Annotation
region helidon
AbstractReadWriteAccessStrategy.java
Annotation
strategy helidon
BaseRegionAccessStrategy.java
Annotation
NonStrictReadWriteCollectionRegionAccessStrategy.java
Annotation
strategy helidon
NonStrictReadWriteEntityRegionAccessStrategy.java
Annotation
strategy helidon
NonStrictReadWriteNaturalIdRegionAccessStrategy.java
Annotation
strategy helidon
ReadOnlyCollectionRegionAccessStrategy.java
Annotation
strategy helidon
ReadOnlyEnt

RedisRunner.java
RedisVersion.java
RedissonRuntimeEnvironment.java
BaseConnectionTest.java
RedissonClusterConnectionRenameTest.java
Annotation
connection helidon
RedissonClusterConnectionTest.java
Annotation
connection helidon
RedissonConnectionTest.java
RedissonMultiConnectionTest.java
RedissonPipelineConnectionTest.java
RedissonSentinelConnectionTest.java
RedissonSubscribeTest.java
BinaryConvertor.java
DataTypeConvertor.java
DistanceConvertor.java
GeoResultsDecoder.java
Annotation
ObjectListReplayDecoder2.java
PointDecoder.java
PropertiesDecoder.java
RedisClusterNodeDecoder.java
RedissonClusterConnection.java
RedissonConnection.java
RedissonConnectionFactory.java
RedissonExceptionConverter.java
RedissonSentinelConnection.java
RedissonSubscription.java
ScoredSortedListReplayDecoder.java
ScoredSortedSetReplayDecoder.java
SecondsConvertor.java
SetReplayDecoder.java
BaseTest.java
ClusterRunner.java
RedisRunner.java
RedisVersion.java
RedissonRuntimeEnvironment.java
BaseConnectionTest.java

RedisClusterNodeDecoder.java
RedissonBaseCommands.java
RedissonBaseReactive.java
RedissonClusterConnection.java
RedissonConnection.java
RedissonConnectionFactory.java
RedissonExceptionConverter.java
RedissonReactiveClusterGeoCommands.java
RedissonReactiveClusterHashCommands.java
RedissonReactiveClusterHyperLogLogCommands.java
RedissonReactiveClusterKeyCommands.java
RedissonReactiveClusterListCommands.java
RedissonReactiveClusterNumberCommands.java
RedissonReactiveClusterServerCommands.java
RedissonReactiveClusterSetCommands.java
RedissonReactiveClusterStreamCommands.java
RedissonReactiveClusterStringCommands.java
RedissonReactiveClusterZSetCommands.java
RedissonReactiveGeoCommands.java
RedissonReactiveHashCommands.java
RedissonReactiveHyperLogLogCommands.java
RedissonReactiveKeyCommands.java
RedissonReactiveListCommands.java
RedissonReactiveNumberCommands.java
RedissonReactivePubSubCommands.java
RedissonReactiveRedisClusterConnection.java
RedissonReactiveRedisConnection.java
RedissonRe

MapWriteBehindTask.java
Annotation
redisson helidon
MapWriterTask.java
Annotation
redisson helidon
PubSubEntry.java
Annotation
PubSubMessageListener.java
Annotation
redisson helidon
PubSubPatternMessageListener.java
Annotation
redisson helidon
PubSubPatternStatusListener.java
Annotation
redisson helidon
PubSubStatusListener.java
Annotation
redisson helidon
QueueTransferService.java
Annotation
redisson helidon
QueueTransferTask.java
Annotation
redisson helidon
RedisClusterNodes.java
Annotation
RedisNodes.java
Annotation
redisnode helidon
Redisson.java
Annotation
redisson helidon
RedissonAtomicDouble.java
Annotation
redisson helidon
RedissonAtomicLong.java
Annotation
redisson helidon
RedissonBaseAdder.java
Annotation
redisson helidon
RedissonBaseLock.java
Annotation
redisson helidon
RedissonBatch.java
Annotation
redisson helidon
RedissonBinaryStream.java
Annotation
redisson helidon
RedissonBitSet.java
Annotation
redisson helidon
RedissonBlockingDeque.java
Annotation
redisson helidon
Redi

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

ListMultiDecoder2.java
Annotation
decoder helidon
ListObjectDecoder.java
Annotation
decoder helidon
ListResultReplayDecoder.java
Annotation
decoder helidon
ListScanResult.java
Annotation
decoder helidon
ListScanResultReplayDecoder.java
Annotation
decoder helidon
Long2MultiDecoder.java
Annotation
decoder helidon
MapCacheScanResult.java
Annotation
decoder helidon
MapCacheScanResultReplayDecoder.java
Annotation
decoder helidon
MapEntriesDecoder.java
Annotation
decoder helidon
MapKeyDecoder.java
Annotation
decoder helidon
MapScanResult.java
Annotation
decoder helidon
MapScanResultReplayDecoder.java
Annotation
decoder helidon
MapValueDecoder.java
Annotation
decoder helidon
MultiDecoder.java
Annotation
decoder helidon
ObjectDecoder.java
Annotation
decoder helidon
ObjectFirstScoreReplayDecoder.java
Annotation
decoder helidon
ObjectListReplayDecoder.java
Annotation
decoder helidon
ObjectMapDecoder.java
Annotation
decoder helidon
ObjectMapEntryReplayDecoder.java
Annotation
decoder helidon
Objec

Annotation
ScheduledParameters.java
Annotation
params helidon
ScheduledWithFixedDelayParameters.java
Annotation
TaskParameters.java
Annotation
params helidon
BaseIterator.java
Annotation
iterator helidon
RedissonBaseIterator.java
Annotation
iterator helidon
RedissonBaseMapIterator.java
Annotation
iterator helidon
RedissonListIterator.java
Annotation
iterator helidon
RedissonMapIterator.java
Annotation
iterator helidon
JCache.java
Annotation
jcache helidon
JCacheEntry.java
Annotation
jcache helidon
JCacheEntryEvent.java
Annotation
jcache helidon
JCacheEventCodec.java
Annotation
jcache helidon
JCacheMBeanServerBuilder.java
Annotation
JCacheManager.java
Annotation
jcache helidon
JCachingProvider.java
Annotation
jcache helidon
JMutableEntry.java
Annotation
jcache helidon
EmptyStatisticsMXBean.java
Annotation
bean helidon
JCacheManagementMXBean.java
Annotation
bean helidon
JCacheStatisticsMXBean.java
Annotation
bean helidon
JCacheConfiguration.java
Annotation
configuration helidon
RedissonC

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



redisson helidon
RedissonRuntimeEnvironment.java
RedissonScoredSortedSetReactiveTest.java
Annotation
redisson helidon
RedissonScoredSortedSetTest.java
Annotation
redisson helidon
RedissonScriptReactiveTest.java
Annotation
redisson helidon
RedissonScriptTest.java
Annotation
redisson helidon
RedissonSemaphoreTest.java
Annotation
redisson helidon
RedissonSetCacheReactiveTest.java
Annotation
redisson helidon
RedissonSetCacheTest.java
Annotation
redisson helidon
RedissonSetMultimapCacheTest.java
Annotation
redisson helidon
RedissonSetMultimapTest.java
Annotation
redisson helidon
RedissonSetReactiveTest.java
Annotation
redisson helidon
RedissonSetTest.java
Annotation
redisson helidon
RedissonSortedSetTest.java
Annotation
redisson helidon
RedissonSpinLockTest.java
Annotation
redisson helidon
RedissonStreamTest.java
RedissonTest.java
Annotation
redisson helidon
RedissonTimeSeriesReactiveTest.java
Annotation
redisson helidon
RedissonTimeSeriesTest.java
Annotation
redisson helidon
RedissonTopicP

0

In [172]:
print(len(refactor_arr))
refactor_arr

1135


['Move RedissonExtensionTest.java to 0',
 'Move JndiRedissonRegionFactory.java to 0',
 'Move RedissonRegionFactory.java to 0',
 'Move BaseRegion.java to 0',
 'Move RedissonCollectionRegion.java to 0',
 'Move RedissonEntityRegion.java to 0',
 'Move RedissonNaturalIdRegion.java to 0',
 'Move RedissonQueryRegion.java to 0',
 'Move RedissonTimestampsRegion.java to 0',
 'Move AbstractReadWriteAccessStrategy.java to 0',
 'Move NonStrictReadWriteCollectionRegionAccessStrategy.java to 0',
 'Move NonStrictReadWriteEntityRegionAccessStrategy.java to 0',
 'Move NonStrictReadWriteNaturalIdRegionAccessStrategy.java to 0',
 'Move ReadOnlyCollectionRegionAccessStrategy.java to 0',
 'Move ReadOnlyEntityRegionAccessStrategy.java to 0',
 'Move ReadOnlyNaturalIdRegionAccessStrategy.java to 0',
 'Move ReadWriteCollectionRegionAccessStrategy.java to 0',
 'Move ReadWriteEntityRegionAccessStrategy.java to 0',
 'Move ReadWriteNaturalIdRegionAccessStrategy.java to 0',
 'Move TransactionalCollectionRegionAccess

In [173]:
temp_dataframe = pd.DataFrame(main_data_arr)
temp_columns = list(ck_metrics.columns)
temp_columns.append('num_dependency')
temp_columns.append('num_line_affected')
temp_dataframe.columns = temp_columns

## For the groupby operation below
temp_dataframe['placeholder_key'] = 'placeholder_key'

In [174]:
temp_dataframe.head()

,file,class,type,cbo,wmc,dit,rfc,lcom,totalMethods,staticMethods,...,maxNestedBlocks,anonymousClassesQty,subClassesQty,lambdasQty,uniqueWordsQty,modifiers,class_name,num_dependency,num_line_affected,placeholder_key
0,C:\Users\tanji\Desktop\SoftwareRemodularization\raw_sourcecode\Red...,org.redisson.helidon.RedissonExtension,class,9,7,1,9,0,2,0,...,1,0,0,2,35,1,RedissonExtension.java,10.0,90,placeholder_key
1,C:\Users\tanji\Desktop\SoftwareRemodularization\raw_sourcecode\Red...,org.redisson.RedissonExtensionTest,class,12,3,1,15,3,3,0,...,0,0,0,0,22,1,RedissonExtensionTest.java,10.0,141,placeholder_key
2,C:\Users\tanji\Desktop\SoftwareRemodularization\raw_sourcecode\Red...,org.redisson.hibernate.JndiRedissonRegionFactory,class,4,6,2,4,1,2,0,...,3,0,0,0,56,1,JndiRedissonRegionFactory.java,10.0,211,placeholder_key
3,C:\Users\tanji\Desktop\SoftwareRemodularization\raw_sourcecode\Red...,org.redisson.hibernate.RedissonRegionFactory,class,23,29,1,27,19,14,0,...,5,0,0,0,115,1,RedissonRegionFactory.java,10.0,448,placeholder_key
4,C:\Users\tanji\Desktop\SoftwareRemodularization\raw_sourcecode\Red...,org.redisson.hibernate.RedissonStrategyRegistrationProvider,class,5,1,1,3,0,1,0,...,0,0,0,0,11,1,RedissonStrategyRegistrationProvider.java,69.0,490,placeholder_key


In [175]:
project_refactoring_data_agg = temp_dataframe.groupby(['placeholder_key'], as_index=False).agg({
       'cbo': ['mean', 'min', 'max','std'], 
        'wmc': ['mean', 'min', 'max','std'], 
        'dit': ['mean', 'min', 'max','std'], 
        'rfc': ['mean', 'min', 'max','std'], 
        'lcom': ['mean', 'min', 'max','std'],
        'totalMethods': ['mean', 'min', 'max','std'], 
        'staticMethods': ['mean', 'min', 'max','std'],
        'publicMethods': ['mean', 'min', 'max','std'], 
        'privateMethods': ['mean', 'min', 'max','std'],
        'protectedMethods': ['mean', 'min', 'max','std'], 
        'defaultMethods': ['mean', 'min', 'max','std'], 
        'abstractMethods': ['mean', 'min', 'max','std'], 
        'finalMethods': ['mean', 'min', 'max','std'],
        'synchronizedMethods': ['mean', 'min', 'max','std'],
        'totalFields': ['mean', 'min', 'max','std'], 
        'staticFields': ['mean', 'min', 'max','std'], 
        'publicFields': ['mean', 'min', 'max','std'],
        'privateFields': ['mean', 'min', 'max','std'],
        'protectedFields': ['mean', 'min', 'max','std'],
        'defaultFields': ['mean', 'min', 'max','std'], 
        'finalFields': ['mean', 'min', 'max','std'],
        'synchronizedFields': ['mean', 'min', 'max','std'],
        'nosi': ['mean', 'min', 'max','std'], 
        'loc': ['mean', 'min', 'max','std'], 
        'returnQty': ['mean', 'min', 'max','std'],
        'loopQty': ['mean', 'min', 'max','std'],
        'comparisonsQty': ['mean', 'min', 'max','std'],
        'tryCatchQty': ['mean', 'min', 'max','std'],
        'parenthesizedExpsQty': ['mean', 'min', 'max','std'],
        'stringLiteralsQty': ['mean', 'min', 'max','std'],
        'numbersQty': ['mean', 'min', 'max','std'], 
        'assignmentsQty': ['mean', 'min', 'max','std'],
        'mathOperationsQty': ['mean', 'min', 'max','std'], 
        'variablesQty': ['mean', 'min', 'max','std'], 
        'maxNestedBlocks': ['mean', 'min', 'max','std'],
        'anonymousClassesQty': ['mean', 'min', 'max','std'],
        'subClassesQty': ['mean', 'min', 'max','std'],
        'lambdasQty': ['mean', 'min', 'max','std'],
        'uniqueWordsQty': ['mean', 'min', 'max','std'],
        'modifiers': ['mean', 'min', 'max','std'],
        'num_dependency': ['mean'],
        'num_line_affected': ['mean'],
})

In [176]:
project_refactoring_data_agg

placeholder_key       cbo                            wmc           \
                        mean min  max        std       mean min  max   
0  placeholder_key  9.692969   0  160  14.133905  15.061719   0  342   

                  dit  ... uniqueWordsQty                      modifiers      \
        std      mean  ...           mean min  max        std       mean min   
0  28.09172  1.574219  ...      49.694531   0  349  48.427578  35.644531   0   

                    num_dependency num_line_affected  
    max         std           mean              mean  
0  1025  184.508186      12.002344     105891.132031  

[1 rows x 163 columns]

In [177]:
columns_to_agg = [
    'cbo', 'wmc', 'dit', 'rfc', 'lcom',
       'totalMethods', 'staticMethods', 'publicMethods', 'privateMethods',
       'protectedMethods', 'defaultMethods', 'abstractMethods', 'finalMethods',
       'synchronizedMethods', 'totalFields', 'staticFields', 'publicFields',
       'privateFields', 'protectedFields', 'defaultFields', 'finalFields',
       'synchronizedFields', 'nosi', 'loc', 'returnQty', 'loopQty',
       'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty',
       'stringLiteralsQty', 'numbersQty', 'assignmentsQty',
       'mathOperationsQty', 'variablesQty', 'maxNestedBlocks',
       'anonymousClassesQty', 'subClassesQty', 'lambdasQty', 'uniqueWordsQty',
       'modifiers'
]


project_refactoring_data_agg_column_names = ['placeholder_key']
for element in columns_to_agg:
    project_refactoring_data_agg_column_names.append(element + '_mean')
    project_refactoring_data_agg_column_names.append(element + '_min')
    project_refactoring_data_agg_column_names.append(element + '_max')
    project_refactoring_data_agg_column_names.append(element + '_std')
    
    
project_refactoring_data_agg_column_names.append('num_dependency_mean')
project_refactoring_data_agg_column_names.append('num_line_affected_mean')

In [178]:
project_refactoring_data_agg.columns = project_refactoring_data_agg_column_names

In [179]:
project_refactoring_data_agg

,placeholder_key,cbo_mean,cbo_min,cbo_max,cbo_std,wmc_mean,wmc_min,wmc_max,wmc_std,dit_mean,...,uniqueWordsQty_mean,uniqueWordsQty_min,uniqueWordsQty_max,uniqueWordsQty_std,modifiers_mean,modifiers_min,modifiers_max,modifiers_std,num_dependency_mean,num_line_affected_mean
0,placeholder_key,9.692969,0,160,14.133905,15.061719,0,342,28.09172,1.574219,...,49.694531,0,349,48.427578,35.644531,0,1025,184.508186,12.002344,105891.132031


In [180]:
#project_refactoring_data_agg['wmc_max'] = 10000
#project_refactoring_data_agg['wmc_max']

In [181]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 54 mins
H2O_cluster_timezone:,Asia/Kuala_Lumpur
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.7
H2O_cluster_version_age:,2 months and 22 days
H2O_cluster_name:,H2O_from_python_tanji_luj4gp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.925 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [182]:
#model_path = "models/EffortEstimationModelv2/GBM_5_AutoML_20211120_201618"
model_path = "models/EffortEstimationModelv2/GBM_grid__1_AutoML_20211125_154317_model_2"
saved_model = h2o.load_model(model_path)

In [183]:
project_refactoring_data_agg.drop(['placeholder_key'],axis=1, inplace=True)

In [184]:
project_refactoring_data_agg

,cbo_mean,cbo_min,cbo_max,cbo_std,wmc_mean,wmc_min,wmc_max,wmc_std,dit_mean,dit_min,...,uniqueWordsQty_mean,uniqueWordsQty_min,uniqueWordsQty_max,uniqueWordsQty_std,modifiers_mean,modifiers_min,modifiers_max,modifiers_std,num_dependency_mean,num_line_affected_mean
0,9.692969,0,160,14.133905,15.061719,0,342,28.09172,1.574219,1,...,49.694531,0,349,48.427578,35.644531,0,1025,184.508186,12.002344,105891.132031


In [185]:
prediction_df = h2o.H2OFrame(project_refactoring_data_agg)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [186]:
prediction_df

cbo_mean,cbo_min,cbo_max,cbo_std,wmc_mean,wmc_min,wmc_max,wmc_std,dit_mean,dit_min,dit_max,dit_std,rfc_mean,rfc_min,rfc_max,rfc_std,lcom_mean,lcom_min,lcom_max,lcom_std,totalMethods_mean,totalMethods_min,totalMethods_max,totalMethods_std,staticMethods_mean,staticMethods_min,staticMethods_max,staticMethods_std,publicMethods_mean,publicMethods_min,publicMethods_max,publicMethods_std,privateMethods_mean,privateMethods_min,privateMethods_max,privateMethods_std,protectedMethods_mean,protectedMethods_min,protectedMethods_max,protectedMethods_std,defaultMethods_mean,defaultMethods_min,defaultMethods_max,defaultMethods_std,abstractMethods_mean,abstractMethods_min,abstractMethods_max,abstractMethods_std,finalMethods_mean,finalMethods_min,finalMethods_max,finalMethods_std,synchronizedMethods_mean,synchronizedMethods_min,synchronizedMethods_max,synchronizedMethods_std,totalFields_mean,totalFields_min,totalFields_max,totalFields_std,staticFields_mean,staticFields_min,staticFields_max,staticFields_std,publicFields_mean,publicFields_min,publicFields_max,publicFields_std,privateFields_mean,privateFields_min,privateFields_max,privateFields_std,protectedFields_mean,protectedFields_min,protectedFields_max,protectedFields_std,defaultFields_mean,defaultFields_min,defaultFields_max,defaultFields_std,finalFields_mean,finalFields_min,finalFields_max,finalFields_std,synchronizedFields_mean,synchronizedFields_min,synchronizedFields_max,synchronizedFields_std,nosi_mean,nosi_min,nosi_max,nosi_std,loc_mean,loc_min,loc_max,loc_std,returnQty_mean,returnQty_min,returnQty_max,returnQty_std,loopQty_mean,loopQty_min,loopQty_max,loopQty_std,comparisonsQty_mean,comparisonsQty_min,comparisonsQty_max,comparisonsQty_std,tryCatchQty_mean,tryCatchQty_min,tryCatchQty_max,tryCatchQty_std,parenthesizedExpsQty_mean,parenthesizedExpsQty_min,parenthesizedExpsQty_max,parenthesizedExpsQty_std,stringLiteralsQty_mean,stringLiteralsQty_min,stringLiteralsQty_max,stringLiteralsQty_std,numbersQty_mean,numbersQty_min,numbersQty_max,numbersQty_std,assignmentsQty_mean,assignmentsQty_min,assignmentsQty_max,assignmentsQty_std,mathOperationsQty_mean,mathOperationsQty_min,mathOperationsQty_max,mathOperationsQty_std,variablesQty_mean,variablesQty_min,variablesQty_max,variablesQty_std,maxNestedBlocks_mean,maxNestedBlocks_min,maxNestedBlocks_max,maxNestedBlocks_std,anonymousClassesQty_mean,anonymousClassesQty_min,anonymousClassesQty_max,anonymousClassesQty_std,subClassesQty_mean,subClassesQty_min,subClassesQty_max,subClassesQty_std,lambdasQty_mean,lambdasQty_min,lambdasQty_max,lambdasQty_std,uniqueWordsQty_mean,uniqueWordsQty_min,uniqueWordsQty_max,uniqueWordsQty_std,modifiers_mean,modifiers_min,modifiers_max,modifiers_std,num_dependency_mean,num_line_affected_mean
9.69297,0,160,14.1339,15.0617,0,342,28.0917,1.57422,1,7,0.937845,13.8438,0,254,28.9182,200.168,0,24747,1148.53,11.2234,0,223,19.2386,0.194531,0,12,0.950931,7.23281,0,219,16.2518,0.420312,0,22,1.61741,0.49375,0,27,1.95691,0.0015625,0,1,0.039513,0.0398437,0,6,0.329626,0.0328125,0,10,0.422789,0.00234375,0,2,0.0624805,1.92656,0,330,9.84291,0.344531,0,46,1.93706,0.0617188,0,11,0.471323,1.30859,0,51,2.9368,0.0945312,0,26,0.944745,0,0,0,0,1.0125,0,49,2.64617,0,0,0,0,5.02734,0,239,18.7082,96.6898,3,3040,204.037,5.86016,0,227,16.8191,0.709375,0,25,2.28499,0.963281,0,110,4.62996,0.282813,0,31,1.49108,0.39375,0,92,3.09702,14.1992,0,1585,68.8916,10.7898,0,866,49.4079,10.8922,0,389,29.7258,1.38594,0,99,5.39853,9.80312,0,330,26.6236,0.365625,-1,6,1.25762,0.369531,0,24,1.38579,0.144531,-1,25,0.976768,1.09844,0,53,4.44703,49.6945,0,349,48.4276,35.6445,0,1025,184.508,12.0023,105891


In [187]:
preds = saved_model.predict(prediction_df)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [188]:
print(preds[0])

predict
8.73524


In [189]:
preds

predict
8.73524


In [ ]:
refactor_arr

In [150]:
saved_model

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_1_AutoML_20211125_152544


GLM Model: summary


,,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,gaussian,identity,Ridge ( lambda = 15.561 ),"nlambda = 30, lambda.max = 703.43, lambda.min = 15.561, lambda.1se...",153,153,13,automl_training_py_6_sid_bd1a




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 782.7144852913913
RMSE: 27.97703496247219
MAE: 18.158385363805124
RMSLE: 1.4200762405657579
R^2: 0.053181589282774944
Mean Residual Deviance: 782.7144852913913
Null degrees of freedom: 253
Residual degrees of freedom: 100
Null deviance: 209976.3555647517
Residual deviance: 198809.47926401338
AIC: 2723.1638436837934

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 801.6948318040318
RMSE: 28.31421607256736
MAE: 18.48058510116557
RMSLE: NaN
R^2: 0.030221823164136352
Mean Residual Deviance: 801.6948318040318
Null degrees of freedom: 253
Residual degrees of freedom: 102
Null deviance: 210509.699745404
Residual deviance: 203630.48727822406
AIC: 2725.249693594421

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,18.479473,1.7476318,18.184616,20.428413,19.464485,15.767283,18.552565
1,mean_residual_deviance,798.9942,200.19315,797.7754,994.5134,994.3213,540.84607,667.51495
2,mse,798.9942,200.19315,797.7754,994.5134,994.3213,540.84607,667.51495
3,null_deviance,42101.94,10696.866,42505.09,52736.63,52123.25,28236.047,34908.684
4,r2,0.028139917,0.016799217,0.04259261,0.0382267,0.0078622755,0.011902851,0.040115148
5,residual_deviance,40615.203,10323.186,40686.547,50720.184,50710.387,27583.15,33375.746
6,rmse,28.081219,3.6123714,28.244919,31.535908,31.532862,23.256098,25.83631
7,rmsle,1.3573587,0.0030857364,1.3549855,NaN,1.3562437,1.360847,NaN



Scoring History: 


,,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_xval,deviance_se,alpha,iterations,training_rmse,training_deviance,training_mae,training_r2
0,,2021-11-25 15:25:44,0.000 sec,1,.7E3,154,823.419641,825.678612,92.828596,0.0,NaN,,,,
1,,2021-11-25 15:25:44,0.001 sec,2,.51E3,154,822.297609,824.785196,92.833202,0.0,NaN,,,,
2,,2021-11-25 15:25:44,0.002 sec,3,.37E3,154,820.834759,823.619924,92.836298,0.0,NaN,,,,
3,,2021-11-25 15:25:44,0.002 sec,4,.27E3,154,818.956528,822.123972,92.834760,0.0,NaN,,,,
4,,2021-11-25 15:25:44,0.003 sec,5,.2E3,154,816.596089,820.246967,92.823052,0.0,NaN,,,,
5,,2021-11-25 15:25:44,0.004 sec,6,.14E3,154,813.707403,817.962249,92.791447,0.0,NaN,,,,
6,,2021-11-25 15:25:44,0.004 sec,7,.1E3,154,810.283824,815.288921,92.724436,0.0,NaN,,,,
7,,2021-11-25 15:25:44,0.005 sec,8,.76E2,154,806.364515,812.314883,92.599889,0.0,NaN,,,,
8,,2021-11-25 15:25:44,0.006 sec,9,.55E2,154,802.036204,809.207946,92.386910,0.0,NaN,,,,
9,,2021-11-25 15:25:44,0.007 sec,10,.4E2,154,797.420935,806.214493,92.049630,0.0,NaN,,,,



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,maxNestedBlocks_max,0.269557,1.000000,0.028207
1,maxNestedBlocks_mean,0.210727,0.781753,0.022051
2,totalFields_mean,0.168841,0.626364,0.017668
3,modifiers_std,0.166467,0.617558,0.017419
4,maxNestedBlocks_min,0.164213,0.609196,0.017183
5,privateFields_mean,0.146784,0.544538,0.015360
6,totalFields_max,0.144130,0.534691,0.015082
7,totalFields_min,0.142252,0.527724,0.014885
8,finalFields_mean,0.137994,0.511931,0.014440
9,lambdasQty_mean,0.137637,0.510606,0.014402



See the whole table with table.as_data_frame()


In [107]:
test_df = pd.read_csv('test_effort_model.csv')

In [108]:
test_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,...,num_dependency_std,num_line_affected_mean,num_line_affected_min,num_line_affected_max,num_line_affected_std,actual_num_of_classes_touched_mean,actual_num_of_classes_touched_min,actual_num_of_classes_touched_max,actual_num_of_classes_touched_std,time_taken_final
0,6ad1e4fe77445e8689f6d3975b26e52165c9c3e6,Bin Fan,fanbin103@gmail.com,2021-08-04 02:49:16,apc999,Fix IndexOutOfBoundsException on async cache\n...,df5dcab8bc308dfd2bf650a895865b13120a9866,2021-08-03 16:07:28,10.0,1,...,0.000000,764.000000,764,764,0.000,9.0,9,9,0.000000,10.0
1,8647c6162423b851dda8d10edf4686473d2e95cc,Zac Blanco,zac@alluxio.com,2021-07-15 21:54:27,ZacBlanco,Update and improve conformance of S3 API\n\n##...,79a5e5c78b7dfcdbf8edbd928a2ff59c904d08f8,2021-07-15 18:38:50,3.0,1,...,0.000000,275.000000,275,275,0.000,6.0,6,6,0.000000,3.0
2,3ce52983e6f50bfb7880b5a2cb13a18e4272170b,Zhan Yuan,yuanzhanhku@gmail.com,2021-07-12 06:39:42,yuanzhanhku,Add CacheContext to URIStatus to enable per-re...,cf79c7837c57c83c1b99a1dab53fec25deb2069d,2021-07-10 20:58:13,33.0,1,...,0.000000,115.000000,115,115,0.000,4.5,3,6,2.121320,33.0
3,a4dc54f7dc0333da096aa779dbaa79060c90d1ad,kqhzz,kuangqinghuazz@163.com,2021-06-09 18:09:26,kuszz,Deprecate leader command\n\nFix #13512\n\npr-l...,1356c0b35fd753d1081665b2c8f6b25da62bf2e8,2021-06-09 02:31:45,15.0,1,...,0.000000,61.000000,61,61,0.000,2.0,2,2,0.000000,15.0
4,7fb84094a6075bcef5a103b40adcac4b26b724ca,Jiacheng Liu,jiacheliu3@gmail.com,2021-05-29 00:37:05,jiacheliu3,Refactor MasterWorkerInfo\n\nCurrently all the...,220237085593d731756e24249bcf88a4d6ea5710,2021-05-28 23:28:46,1.0,1,...,93.543216,458.666667,24,1245,682.248,35.0,1,54,29.512709,1.0


In [109]:
test_df.drop(columns=['sha',
'name','email','login',
'date','message',
'parent_sha','parent_date',
'time_taken','contains_refactoring',
'kmean_label','mean',
'min','max',
'project_name','commit_compared_with','actual_num_of_classes_touched_mean',
 'actual_num_of_classes_touched_min',
 'actual_num_of_classes_touched_max',
 'actual_num_of_classes_touched_std','time_taken_final']
             ,inplace=True)

In [110]:
test_df

,cbo_mean,cbo_min,cbo_max,cbo_std,wmc_mean,wmc_min,wmc_max,wmc_std,dit_mean,dit_min,...,modifiers_max,modifiers_std,num_dependency_mean,num_dependency_min,num_dependency_max,num_dependency_std,num_line_affected_mean,num_line_affected_min,num_line_affected_max,num_line_affected_std
0,38.000000,38.0,38.0,0.000000,57.00,57.0,57.0,0.000000,1.000000,1.0,...,17.0,0.000000,322.000000,322.0,322.0,0.000000,764.000000,764,764,0.000000
1,12.000000,12.0,12.0,0.000000,17.00,17.0,17.0,0.000000,1.000000,1.0,...,1.0,0.000000,15.000000,15.0,15.0,0.000000,275.000000,275,275,0.000000
2,4.000000,4.0,4.0,0.000000,15.00,15.0,15.0,0.000000,1.000000,1.0,...,1.0,0.000000,371.000000,371.0,371.0,0.000000,115.000000,115,115,0.000000
3,11.000000,11.0,11.0,0.000000,8.00,8.0,8.0,0.000000,2.000000,2.0,...,17.0,0.000000,85.000000,85.0,85.0,0.000000,61.000000,61,61,0.000000
4,27.666667,3.0,76.0,41.860881,50.00,1.0,140.0,78.044859,1.666667,1.0,...,1.0,0.000000,69.333333,8.0,177.0,93.543216,458.666667,24,1245,682.248000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,77.000000,77.0,77.0,0.000000,260.00,260.0,260.0,0.000000,4.000000,4.0,...,17.0,0.000000,11.000000,11.0,11.0,0.000000,3932.000000,3932,3932,0.000000
362,11.250000,7.0,22.0,6.692213,24.00,4.0,54.0,22.309511,1.500000,1.0,...,17.0,8.552360,105.000000,4.0,321.0,137.772068,217.500000,67,566,217.340418
363,8.500000,2.0,17.0,5.680376,13.25,7.0,19.0,4.753946,1.250000,1.0,...,17.0,0.447214,53.000000,5.0,143.0,58.052849,117.500000,31,198,61.085732
364,7.000000,7.0,7.0,0.000000,2.00,2.0,2.0,0.000000,1.000000,1.0,...,1.0,0.000000,42.000000,42.0,42.0,0.000000,18.000000,18,18,0.000000


In [111]:
prediction_df = h2o.H2OFrame(test_df)
preds = saved_model.predict(prediction_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [112]:
preds

predict
8.80149
17.1904
18.8304
19.1618
6.99672
17.551
17.0801
16.6316
13.7009
13.3708


In [115]:
len(preds)

366

In [116]:
pd_preds = preds.as_data_frame()

In [117]:
pd_preds['predict'].value_counts()

18.775940    2
16.259852    2
4.799652     2
20.734580    2
20.558679    2
            ..
20.994492    1
14.732429    1
18.621463    1
15.743354    1
8.801486     1
Name: predict, Length: 361, dtype: int64